In [1]:
import numpy as np
import pandas as pd 
import os

In [3]:
import cv2 

## Data preprocessing

#### Data Loading
In this section, three main datasets are loaded:
- df_images: Contains the image data for object detection, including file paths for image_id, image_name, and corresponding metadata.
- df_labels: Contains the ground truth labels for object detection, including object_class, bounding_box_coordinates, and image_id.
- df_calibration: Includes calibration parameters such as camera matrix, distortion coefficients, and extrinsic parameters for aligning image and sensor data.

In [4]:
image_path  = "./data_object_image_2/training/image_2/"

In [5]:
# List all image files
image_files = os.listdir(image_path)

# Create a DataFrame to store image file paths
df_images = pd.DataFrame({
    'image_id': range(len(image_files)),
    'image_name': image_files,
    'file_path': [os.path.join(image_path, img) for img in image_files]
})

In [6]:
# Print the first few entries of df_images
print(df_images.head())

   image_id  image_name                                          file_path
0         0  000000.png  ./data_object_image_2/training/image_2/000000.png
1         1  000001.png  ./data_object_image_2/training/image_2/000001.png
2         2  000002.png  ./data_object_image_2/training/image_2/000002.png
3         3  000003.png  ./data_object_image_2/training/image_2/000003.png
4         4  000004.png  ./data_object_image_2/training/image_2/000004.png


In [9]:
# Define the path to your KITTI dataset labels
label_path = "./data_object_label_2/training/label_2/"

# Function to load and parse a label file
def load_labels(label_file):
    labels = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            obj_class = parts[0]
            x_min = int(float(parts[4]))
            y_min = int(float(parts[5]))
            x_max = int(float(parts[6]))
            y_max = int(float(parts[7]))
            labels.append({'object_class': obj_class, 'bounding_box': (x_min, y_min, x_max, y_max)})
    return labels

# List all label files and accumulate label data
label_files = os.listdir(label_path)
label_data = []

for i, label_file in enumerate(label_files):
    file_path = os.path.join(label_path, label_file)
    labels = load_labels(file_path)
    for label in labels:
        label_data.append({
            'image_id': i,
            'object_class': label['object_class'],
            'bounding_box_coordinates': label['bounding_box']
        })

# Create a DataFrame using pd.concat
df_labels = pd.DataFrame(label_data)

# Print the first few entries of df_labels
print(df_labels.head())

   image_id object_class bounding_box_coordinates
0         0   Pedestrian     (712, 143, 810, 307)
1         1        Truck     (599, 156, 629, 189)
2         1          Car     (387, 181, 423, 203)
3         1      Cyclist     (676, 163, 688, 193)
4         1     DontCare     (503, 169, 590, 190)


In [12]:
# Define the path to your KITTI dataset calibration files
calibration_path = "./data_object_calib/training/calib/"

# Function to load calibration parameters
def load_calibration(calib_file):
    calib_data = {}
    with open(calib_file, 'r') as f:
        for line in f:
            line = line.strip()
            
            # Skip empty lines or comments
            if not line:
                continue

            # Check if the line has the expected ':' delimiter
            if ':' in line:
                key, value = line.split(':', 1)
                calib_data[key.strip()] = [float(x) for x in value.strip().split()]
            else:
                print(f"Skipping malformed line: {line}")
    
    return calib_data

# List all calibration files and accumulate calibration data
calibration_files = os.listdir(calibration_path)
calibration_data = []

for i, calib_file in enumerate(calibration_files):
    file_path = os.path.join(calibration_path, calib_file)
    calib_data = load_calibration(file_path)
    calibration_data.append({
        'image_id': i,
        'calibration_data': calib_data
    })

# Create a DataFrame using pd.concat
df_calibration = pd.DataFrame(calibration_data)

# Print the first few entries of df_calibration
print(df_calibration.head())

   image_id                                   calibration_data
0         0  {'P0': [707.0493, 0.0, 604.0814, 0.0, 0.0, 707...
1         1  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...
2         2  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...
3         3  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...
4         4  {'P0': [721.5377, 0.0, 609.5593, 0.0, 0.0, 721...
